In [1]:
# 1) Spark session + JDBC
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

spark = (
    SparkSession.builder
    .appName("Transfermarkt ML - latest_fee")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
            "org.postgresql:postgresql:42.7.3")
    .getOrCreate()
)

jdbc_url = "jdbc:postgresql://postgres:5432/metastore"
jdbc_props = {"user": "hive", "password": "metastore", "driver": "org.postgresql.Driver"}
print("Spark:", spark.version)


Spark: 3.5.0


In [2]:
# 2) Tạo base dataset: per-player features + latest transfer
sql_base = """
WITH latest_tx AS (
  SELECT
    ft.player_key,
    ft.fee,
    ft.transfer_date,
    ft.club_to_key,
    ft.club_from_key,
    ft.season,
    ROW_NUMBER() OVER (
      PARTITION BY ft.player_key
      ORDER BY ft.transfer_date DESC NULLS LAST, ft.fee DESC
    ) rn
  FROM public.fact_transfers ft
),
perf_agg AS (
  SELECT
    fp.player_key,
    COALESCE(SUM(fp.goals), 0)            AS goals,
    COALESCE(SUM(fp.assists), 0)          AS assists,
    COALESCE(SUM(fp.minutes_played), 0)   AS minutes_played
  FROM public.fact_performance fp
  GROUP BY fp.player_key
)
SELECT
  dp.player_key,
  dp.name,
  dp.position,
  dp.age,
  pa.goals,
  pa.assists,
  pa.minutes_played,
  CASE WHEN pa.minutes_played > 0 THEN pa.goals   * 90.0 / pa.minutes_played END AS goals_per90,
  CASE WHEN pa.minutes_played > 0 THEN pa.assists * 90.0 / pa.minutes_played END AS assists_per90,
  lt.fee        AS latest_fee,
  lt.season     AS latest_season,
  lt.club_to_key,
  lt.club_from_key
FROM public.dim_players dp
LEFT JOIN perf_agg  pa ON dp.player_key = pa.player_key
LEFT JOIN latest_tx lt ON dp.player_key = lt.player_key AND lt.rn = 1
"""

# Spark hỗ trợ option("query", ...) (Spark 3+). Nếu bản bạn không hỗ trợ, dùng dbtable="( ... ) t".
base_df = (spark.read.format("jdbc")
           .option("url", jdbc_url)
           .option("query", sql_base)
           .options(**jdbc_props)
           .load())

print("Base rows:", base_df.count())
base_df.printSchema()


Base rows: 32601
root
 |-- player_key: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- position: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- goals: long (nullable = true)
 |-- assists: long (nullable = true)
 |-- minutes_played: long (nullable = true)
 |-- goals_per90: decimal(38,18) (nullable = true)
 |-- assists_per90: decimal(38,18) (nullable = true)
 |-- latest_fee: double (nullable = true)
 |-- latest_season: string (nullable = true)
 |-- club_to_key: integer (nullable = true)
 |-- club_from_key: integer (nullable = true)



In [3]:
# 3) Clean: lọc cầu thủ có mẫu đủ lớn, fee > 0; điền Null -> 0 cho per90
df = (base_df
  	.filter((F.col("minutes_played") >= 900) & (F.col("latest_fee").isNotNull()) & (F.col("latest_fee") > 0))
  	.fillna({"goals_per90": 0.0, "assists_per90": 0.0})
 	)
 
# Đặc trưng bổ sung từ insight BI
df = df.withColumn("age_sq", F.col("age") * F.col("age")) \
   	.withColumn("age_x_goals90", F.col("age") * F.col("goals_per90"))


In [5]:
# 4) Tính prestige: tổng chi (mua) & tổng thu (bán) theo CLB

# Đọc bảng transfers
ft = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .options(**jdbc_props) \
    .option("dbtable", "public.fact_transfers") \
    .load()

# buyers: club_to_key -> spend_total, buys
buyers = ft.where(F.col("fee") > 0) \
    .groupBy("club_to_key") \
    .agg(
        F.sum("fee").alias("spend_total"),
        F.count("*").alias("buys")
    ) \
    .withColumnRenamed("club_to_key", "club_key_buy")

# sellers: club_from_key -> revenue_total, sells
sellers = ft.where(F.col("fee") > 0) \
    .groupBy("club_from_key") \
    .agg(
        F.sum("fee").alias("revenue_total"),
        F.count("*").alias("sells")
    ) \
    .withColumnRenamed("club_from_key", "club_key_sell")

# full_outer join theo key tương đương
prestige = buyers.join(
    sellers,
    buyers.club_key_buy == sellers.club_key_sell,
    "full_outer"
).select(
    # Lấy club key thống nhất
    F.coalesce(buyers.club_key_buy, sellers.club_key_sell).alias("club_key"),

    # Lấy spend_total, nếu null thì 0
    F.coalesce(buyers.spend_total, F.lit(0.0)).alias("spend_total"),
    F.coalesce(buyers.buys, F.lit(0)).alias("buys"),

    # Lấy revenue_total, nếu null thì 0
    F.coalesce(sellers.revenue_total, F.lit(0.0)).alias("revenue_total"),
    F.coalesce(sellers.sells, F.lit(0)).alias("sells")
)

# 5) Min-max scaling
stats = prestige.agg(
    F.min("spend_total").alias("min_sp"),
    F.max("spend_total").alias("max_sp"),
    F.min("revenue_total").alias("min_rev"),
    F.max("revenue_total").alias("max_rev")
).collect()[0]

min_sp, max_sp = stats["min_sp"], stats["max_sp"]
min_rev, max_rev = stats["min_rev"], stats["max_rev"]

prestige = prestige \
    .withColumn(
        "prestige_buy",
        F.when(F.lit(max_sp) == F.lit(min_sp), F.lit(0.0))
         .otherwise((F.col("spend_total") - F.lit(min_sp)) / F.lit(max_sp - min_sp))
    ) \
    .withColumn(
        "prestige_sell",
        F.when(F.lit(max_rev) == F.lit(min_rev), F.lit(0.0))
         .otherwise((F.col("revenue_total") - F.lit(min_rev)) / F.lit(max_rev - min_rev))
    ) \
    .withColumn(
        "prestige_overall",
        0.7 * F.col("prestige_buy") + 0.3 * F.col("prestige_sell")
    )

# 6) Join vào df chính theo club_to_key
df = df.join(
    prestige.select("club_key", "prestige_overall"),
    df.club_to_key == prestige.club_key,
    "left"
).drop("club_key")

# Nếu prestige_overall null -> 0
df = df.fillna({"prestige_overall": 0.0})


In [7]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
 
# 5) Column chọn làm feature
num_feats = ["age", "age_sq", "goals_per90", "assists_per90", "minutes_played",
         	"prestige_overall", "age_x_goals90"]
cat_feat  = "position"   # one-hot
 
# 6) Encode position
idx = StringIndexer(inputCol=cat_feat, outputCol=f"{cat_feat}_idx", handleInvalid="keep")
ohe = OneHotEncoder(inputCols=[f"{cat_feat}_idx"], outputCols=[f"{cat_feat}_ohe"], dropLast=True)
 
# 7) Assemble vector (định nghĩa, chưa chạy)
assembler = VectorAssembler(
    inputCols=num_feats + [f"{cat_feat}_ohe"],
    outputCol="features",
    handleInvalid="keep"
)

# 8) Tạo cột label (log1p)
df_ml = df.withColumn("label", F.log1p("latest_fee"))

# 9) Chia tập
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

# 10) Models
lr = LinearRegression(featuresCol="features", labelCol="label", elasticNetParam=0.2, regParam=0.05, standardization=True)
rf = RandomForestRegressor(featuresCol="features", labelCol="label", numTrees=200, maxDepth=10, seed=42)

# 11) Pipelines
pipe_lr = Pipeline(stages=[idx, ohe, assembler, lr])
pipe_rf = Pipeline(stages=[idx, ohe, assembler, rf])

# 12) Train
m_lr = pipe_lr.fit(train_df)
m_rf = pipe_rf.fit(train_df)
 
# 13) Predict
pred_lr = m_lr.transform(test_df).withColumn("pred_fee", F.expm1("prediction"))
pred_rf = m_rf.transform(test_df).withColumn("pred_fee", F.expm1("prediction"))
 
# 14) Evaluate (trên log và fee gốc)
ev_r2  = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
ev_rmse= RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
 
print("LR  R2 (log):", ev_r2.evaluate(pred_lr))
print("LR RMSE(log):", ev_rmse.evaluate(pred_lr))
print("RF  R2 (log):", ev_r2.evaluate(pred_rf))
print("RF RMSE(log):", ev_rmse.evaluate(pred_rf))
 
# RMSE trên thang phí gốc (ước lượng): tính RMSE của pred_fee vs latest_fee
from pyspark.sql.functions import col, pow, sqrt, avg
rmse_lr = pred_lr.select(sqrt(avg(pow(col("pred_fee") - col("latest_fee"), 2))).alias("rmse_fee")).first()["rmse_fee"]
rmse_rf = pred_rf.select(sqrt(avg(pow(col("pred_fee") - col("latest_fee"), 2))).alias("rmse_fee")).first()["rmse_fee"]
print("LR RMSE (fee gốc):", rmse_lr)
print("RF RMSE (fee gốc):", rmse_rf)


LR  R2 (log): 0.3141569403030172
LR RMSE(log): 1.333572588645431
RF  R2 (log): 0.30167910019903155
RF RMSE(log): 1.3456490403108199
LR RMSE (fee gốc): 13939933.2833905
RF RMSE (fee gốc): 12765062.241278186


In [8]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
 
paramGrid = (ParamGridBuilder()
         	.addGrid(lr.regParam, [0.0, 0.01, 0.05, 0.1])
         	.addGrid(lr.elasticNetParam, [0.0, 0.2, 0.5, 0.8, 1.0])  # 0=L2, 1=L1
         	.build())
 
cv = CrossValidator(estimator=pipe_lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2"),
                	numFolds=5, parallelism=2)
 
m_lr_cv = cv.fit(train_df)
pred_lr_cv = m_lr_cv.transform(test_df).withColumn("pred_fee", F.expm1("prediction"))
print("LR (CV) R2 (log):", ev_r2.evaluate(pred_lr_cv))


LR (CV) R2 (log): 0.3085714180109085


In [9]:
# Lấy model cuối cùng trong pipeline LR
lr_model = m_lr.stages[-1]  # LinearRegressionModel

coef = lr_model.coefficients
intercept = lr_model.intercept

print("Intercept:", intercept)
print("Num features:", len(coef))

# Ánh xạ tên feature ↔ hệ số
feat_names = assembler.getInputCols()
for name, c in zip(feat_names, coef):
    print(f"{name}: {c}")


Intercept: 16.975541851810227
Num features: 11
age: -0.07245644481358803
age_sq: -0.0021296709364196577
goals_per90: 1.4469129515582146
assists_per90: 1.0043923939592263
minutes_played: 0.0001165838782658863
prestige_overall: 36.56100424837787
age_x_goals90: 0.0
position_ohe: 0.06072075192967705


In [10]:
rf_model = m_rf.stages[-1]  # RandomForestRegressionModel
importances = rf_model.featureImportances.toArray()
feat_names = assembler.getInputCols()

# Sắp xếp top 10 feature quan trọng nhất
pairs = sorted(zip(feat_names, importances), key=lambda x: x[1], reverse=True)
for name, score in pairs[:10]:
    print(f"{name}: {score}")


minutes_played: 0.3528378473070338
age: 0.14815968635461313
assists_per90: 0.11690636520681487
goals_per90: 0.09895701039058205
age_sq: 0.09593725393534952
age_x_goals90: 0.09302035546779026
prestige_overall: 0.050235588036173236
position_ohe: 0.012042361395226953


In [11]:
# Lưu Linear Regression model
m_lr.write().overwrite().save("/home/jovyan/work/models/latest_fee_lr")
 
# Lưu Random Forest model
m_rf.write().overwrite().save("/home/jovyan/work/models/latest_fee_rf")
 
print("  Saved both LR & RF models.")

  Saved both LR & RF models.


In [12]:
# Tạo cột dự đoán (pred_fee) từ mô hình Random Forest (hoặc Linear cũng được)
pred_rf_full = m_rf.transform(df) \
	.withColumn("pred_fee", F.expm1("prediction")) \
	.select("player_key", "name", "position", "latest_fee", "pred_fee")  # giữ cột cần thiết
 
pred_rf_full.show(10)


+----------+-----------------+----------+----------+------------------+
|player_key|             name|  position|latest_fee|          pred_fee|
+----------+-----------------+----------+----------+------------------+
|     17575|      myrto uzuni|    Attack| 4000000.0| 2713297.843587306|
|     18265|       lucas boyé|    Attack| 7000000.0| 4281333.527637985|
|     20807|      sergio ruiz|  Midfield| 1000000.0| 895186.9310708274|
|     21009|   gonzalo villar|  Midfield| 1800000.0| 1796018.690053673|
|     21029|    martin hongla|  Midfield| 2700000.0| 2580514.592509783|
|     16877|  mikkel kallesøe|  Defender|  335000.0| 485545.4411276593|
|     19938|      ivor pandur|Goalkeeper| 2000000.0|1864514.6596754778|
|     25313|  million manhoef|    Attack| 4000000.0| 1519616.356680397|
|     29042|eric-junior bocat|  Defender| 1200000.0|1236471.3835932168|
|       645|    jonas hofmann|  Midfield|     1.0E7| 6935216.866136171|
+----------+-----------------+----------+----------+------------

In [13]:
# Tạo cột dự đoán (pred_fee) từ mô hình Random Forest 
pred_rf_full = m_rf.transform(df) \
	.withColumn("pred_fee", F.expm1("prediction")) \
	.select("player_key", "name", "position", "latest_fee", "pred_fee")  # giữ cột cần thiết
 
pred_rf_full.show(10)


+----------+-----------------+----------+----------+------------------+
|player_key|             name|  position|latest_fee|          pred_fee|
+----------+-----------------+----------+----------+------------------+
|     17575|      myrto uzuni|    Attack| 4000000.0| 2713297.843587306|
|     18265|       lucas boyé|    Attack| 7000000.0| 4281333.527637985|
|     20807|      sergio ruiz|  Midfield| 1000000.0| 895186.9310708274|
|     21009|   gonzalo villar|  Midfield| 1800000.0| 1796018.690053673|
|     21029|    martin hongla|  Midfield| 2700000.0| 2580514.592509783|
|     16877|  mikkel kallesøe|  Defender|  335000.0| 485545.4411276593|
|     19938|      ivor pandur|Goalkeeper| 2000000.0|1864514.6596754778|
|     25313|  million manhoef|    Attack| 4000000.0| 1519616.356680397|
|     29042|eric-junior bocat|  Defender| 1200000.0|1236471.3835932168|
|       645|    jonas hofmann|  Midfield|     1.0E7| 6935216.866136171|
+----------+-----------------+----------+----------+------------

In [14]:
top50 = (pred_rf_full
     	.orderBy(F.desc("pred_fee"))
     	.limit(50)
     	.toPandas())
 
# Lưu file CSV
top50.to_csv("/home/jovyan/work/models/top50_pred.csv", index=False)
 
print("  Saved top 50 highest predicted players to CSV.")


  Saved top 50 highest predicted players to CSV.
